<h1><center>OPTUNA vs БАЙЕС</center></h1>

Вернёмся к анализу датасета «Космолайна» об оттоке клиентов. До этого вы применяли к нему методы Grid Search и Random Search для оптимизации гиперпараметров модели. Новая задача — использовать байесовский подход. Вы будете работать с алгоритмом TPE. 

Как и до этого, вам предстоит обучить новую версию модели — но в этот раз интегрировав процесс обучения с библиотекой optuna для оптимизации гиперпараметров. Так, вы не только автоматизируете поиск наилучших настроек для модели, но и сделаете его более точным и целенаправленным. 

Вы также потренируетесь проводить интеграцию с MLflow. Напомним, что это позволяет систематизировать и сохранять информацию обо всех экспериментах, проведённых в ходе подбора гиперпараметров. И к тому же поможет анализировать результаты.

Не забудем и про воспроизводимость. Для этого вы сохраните всю информацию о процессе подбора гиперпараметров в локальную базу данных. В реальных проектах это будет отличной страховкой, ведь вы сможете в любой момент восстановить условия эксперимента, повторить его и проверить результаты.

Так начнём же.

---

Импортируем библиотеки и настроим параметры:

In [ ]:
import pandas as pd
import numpy as np
import os
import psycopg
import mlflow
from mlflow.tracking import MlflowClient
from catboost import CatBoostClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import ( OneHotEncoder, SplineTransformer, QuantileTransformer, StandardScaler,
                                    RobustScaler, PolynomialFeatures, KBinsDiscretizer )
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from autofeat import AutoFeatRegressor, AutoFeatClassifier

import catboost as cb
from catboost import CatBoostClassifier
from category_encoders import CatBoostEncoder

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

from optuna.integration.mlflow import MLflowCallback

TABLE_NAME = "clean_users_churn" # таблица с данными
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

EXPERIMENT_NAME = "model_bayesian_search" # ваш код здесь
RUN_NAME = "model_bayesian_search"
REGISTRY_MODEL_NAME = 'model_bayesian_search' # ваш код здесь

STUDY_DB_NAME = "sqlite:///local.study.db"
STUDY_NAME = "churn_model"

experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)\
    if not mlflow.get_experiment_by_name(EXPERIMENT_NAME)\
    else mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

Загрузим таблицу:

In [2]:
connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {"host": 'rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net', #os.getenv("DB_DESTINATION_HOST"),
                        "port": '6432', #os.getenv("DB_DESTINATION_PORT"),
                        "dbname": 'playground_mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_NAME"),
                        "user": 'mle_20250529_05fed48463', #os.getenv("DB_DESTINATION_USER"),
                        "password": '0c567edd8ad8472e87d5c85cc4d664e4' } #os.getenv("DB_DESTINATION_PASSWORD")}
connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)
df['target'] = (df['end_date'].notna()).astype(int)
df.head(2) 

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,1,7590-VHVEG,2020-01-01,NaT,Month-to-month,Yes,Electronic check,29.85,29.85,DSL,...,No,No,No,No,Female,0,Yes,No,No,0
1,2,5575-GNVDE,2017-04-01,NaT,One year,No,Mailed check,56.95,1889.50,DSL,...,Yes,No,No,No,Male,0,No,No,No,0


Подготовим данные для обучения:

In [3]:
# Выделим признаки в три отдельные таблицы для дальнейшей работы:
features = df.drop(['customer_id','target'],axis=1)
num_features = features.select_dtypes(include=['float', 'int'])
date_features = features.select_dtypes(include='datetime64[ns]')
cat_features = features.select_dtypes(include='object')

# Посчитаем колво уникальных значений для катег. переменных и создадим создадим два датасета:
unique_values_per_col = cat_features.nunique().value_counts()
binary_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()==2] ]
other_cat_features = cat_features[ [i for i in cat_features.columns if cat_features[i].nunique()!=2] ]

# Бинарные подразделяем на два - "да/нет" и другие бинарные:
yes_no_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()==True] ]
other_binary_features = binary_cat_features[ [i for i in binary_cat_features.columns if\
binary_cat_features[i].isin(['Yes','yes','No','no',None,np.nan]).all()!=True] ]

# Дубликаты
is_duplicated_id = df.duplicated(subset=['customer_id'], keep=False)

# Пропуски
cols_with_nans = df.isnull().sum()
cols_with_nans = cols_with_nans[cols_with_nans > 0].index.drop('end_date')
for col in cols_with_nans:
    if df[col].dtype in [float, int]:
        fill_value = df[col].mean()
    elif df[col].dtype == 'object':
        fill_value = df[col].mode().iloc[0]
    df[col] = df[col].fillna(fill_value)

# Выбросы
num_cols = df.select_dtypes(['float']).columns
threshold = 1.5
potential_outliers = pd.DataFrame()
for col in num_cols:
	Q1 = df[col].quantile(0.25)
	Q3 = df[col].quantile(0.75)
	IQR = Q3 - Q1
	margin = threshold * IQR
	lower = Q1 - margin
	upper = Q3 + margin
	potential_outliers[col] = ~df[col].between(lower, upper)
outliers = potential_outliers.any(axis=1)

df.drop(columns=['id', 'customer_id', 'begin_date', 'end_date'], inplace=True)
df[-3:]

,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
7016,Month-to-month,Yes,Electronic check,29.60,346.45,DSL,Yes,No,No,No,No,No,Female,0,Yes,Yes,No,0
7017,Month-to-month,Yes,Mailed check,74.40,306.60,Fiber optic,No,No,No,No,No,No,Male,1,Yes,No,Yes,1
7018,Two year,Yes,Bank transfer (automatic),105.65,6844.50,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Male,0,No,No,No,0


Первично обучим модель:

In [4]:
features = ["monthly_charges", "total_charges", "senior_citizen"]
target = "target"

split_column = "monthly_charges" # ваш код здесь
stratify_column = target # ваш код здесь
test_size = 0.2 # ваш код здесь

df = df.sort_values(by=[split_column])

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target],\
                                   test_size=test_size, shuffle=False) # ваш код здесь

print(f"Размер выборки для обучения: {X_train.shape}")
print(f"Размер выборки для теста: {X_test.shape}")

Размер выборки для обучения: (5615, 3)
Размер выборки для теста: (1404, 3)


И теперь применим optuna:

In [8]:
def objective(trial: optuna.Trial) -> float:
    param = { "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
              "depth": trial.suggest_int("depth", 1, 12),
              "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.1, 5),
              "random_strength": trial.suggest_float("random_strength", 0.1, 5),
              "loss_function": "Logloss",
              "task_type": "CPU",
              "random_seed": 0,
              "iterations": 300,
              "verbose": False } # ваш код здесь #
    model = CatBoostClassifier(**param)

    skf = StratifiedKFold(n_splits=2) # ваш код здесь #)

    metrics = defaultdict(list)
    for i, (train_index, val_index) in enumerate(skf.split(X_train, y_train)):
        # ваш код здесь #
        train_x = X_train.iloc[train_index]  # Добавление train_x
        train_y = y_train.iloc[train_index]
        val_x = X_train.iloc[val_index] # Добавление val_x 
        val_y = y_train.iloc[val_index]
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        model.fit(train_x, train_y)  # Использование train_x для обучения модели
        probas = model.predict_proba(val_x)[:, 1]
        prediction = model.predict(val_x)

        _, err1, _, err2 = confusion_matrix(val_y, prediction, normalize='all').ravel()
        auc = roc_auc_score(val_y, probas)
        precision = precision_score(val_y, prediction)
        recall = recall_score(val_y, prediction)
        f1 = f1_score(val_y, prediction)
        logloss = log_loss(val_y, prediction)
        
        metrics["err1"].append(err1)
        metrics["err2"].append(err2)
        metrics["auc"].append(auc)
        metrics["precision"].append(precision)
        metrics["recall"].append(recall)
        metrics["f1"].append(f1)
        metrics["logloss"].append(logloss)


    # ваш код здесь #
    err1 = sum(metrics["err1"]) / len(metrics["err1"])
    err_1 = median(array(metrics['err1']))
    err2 = sum(metrics["err2"]) / len(metrics["err2"])
    err_2 = median(array(metrics['err2']))
    auc = median(array(metrics['auc']))
    precision = median(array(metrics['precision']))
    recall = median(array(metrics['recall']))
    f1 = median(array(metrics['f1']))
    logloss = median(array(metrics['logloss']))

    return auc



Залогируем всё это:

In [19]:

os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = 'YCAJE3Nlz8iDILW5VTYM1ihQB' #os.getenv("S3_ACCESS_KEY")
os.environ["AWS_SECRET_ACCESS_KEY"] = 'YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44' #os.getenv("S3_SECRET_KEY")
os.environ['MLFLOW_ARTIFACT_URI'] = 'http://s3-student-mle-20250529-05fed48463'

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if not experiment:
    experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
else:
    experiment_id = experiment.experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id


In [20]:

mlflc = MlflowCallback(tracking_uri=f'http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}',
metric_name='AUC', create_experiment=False, experiment_name=EXPERIMENT_NAME,
mlflow_kwargs={'experiment_id': experiment_id, 'tags': {MLFLOW_PARENT_RUN_ID: run_id}}) # ваш код здесь #

study = optuna.create_study(study_name=STUDY_NAME, storage=STUDY_DB_NAME,
sampler=optuna.samplers.TPESampler(), direction='maximize') # ваш код здесь #
study.optimize(objective, n_trials=10, callbacks=[mlflc]) # ваш код здесь #)
best_params = study.best_params # ваш код здесь #

print(f"Number of finished trials: {len(study.trials)}")
print(f"Best params: {best_params}")

NameError: name 'MlflowCallback' is not defined